In [42]:
import yaml
import hnswlib
import pickle
import json
from src.utils import clean, normalize, remove_stop_words
from src.config import token # токен в локальном файле в целях безопасности
from gensim.models import FastText
import numpy as np
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters

In [3]:
NUM_ELEMENTS = 154536
DIM = 100
MAX_WORDS = 50

In [4]:
ft = FastText.load('/mnt/f/data/bot/ft.model')

In [5]:
p = hnswlib.Index(space ='cosine', dim=DIM)

In [6]:
p.load_index('/mnt/f/data/bot/index.bin', max_elements=NUM_ELEMENTS)

In [7]:
with open('/mnt/f/data/bot/tfidf.pkl', 'rb') as file:
    tfidf = pickle.load(file)

In [8]:
with open('/mnt/f/data/bot/answers.json', 'r') as file:
    answers = json.load(file)

In [9]:
def vectorize_sent(tokens, model, tfidf):
    vector = np.zeros(model.vector_size)
    n_tokens = len(tokens)
    weight_sum = 0
    tfidf_feat = tfidf.get_feature_names()
    final_tf_idf = tfidf.transform([' '.join(tokens)])
    
    if not n_tokens:
        return vector
    
    for token in tokens:
        try:
            weight = final_tf_idf[0, tfidf_feat.index(token)]
        except:
            weight = 0
        vector += (model.wv.get_vector(token) * weight)
        weight_sum += weight
        
    if not weight_sum:
        return vector * 0
        
    vector /= weight_sum
    
    return vector

In [10]:
def get_answer_candidate(question: str, model=ft, tfidf=tfidf):
    tokens = normalize(clean(question))
    tokens_vect = vectorize_sent(tokens, model, tfidf)
    label, distance = p.knn_query(tokens_vect, k=1)
    return label[0][0], distance[0][0]

In [63]:
def get_answer(question: str, answers=answers):
    label, distance = get_answer_candidate(question)
    if distance > 0.3:
        return 'Ответ не найден!'
    else:
        answer_candidates = answers[str(label)]
        # если ответов несколько, то выбираем случайный
        if len(answer_candidates) > 1:
            return answer_candidates[np.random.choice(len(answer_candidates))]
        else:
            return answer_candidates[0]

In [12]:
get_answer('Кто такие вампиры?')

'В природе - вид кровососущих летучих мышей, живущих в Южной Америке. В литературе - все зависит от фантазии автора - у Анны Райс они одни, у Сергея Лукьяненко - другие. , у Уильяма Тенна - третьи, у Роберта Блоха - четвертые, у Брема Стокера - пятые. Единственное общее то что они потребляют человеческую кровь.. \n'

In [72]:
get_answer('Кто такие вампиры?')

'вампиры очень бледные кровососущие твари!!! =))))). \n'

In [67]:
get_answer('Как починить кран?')

'У меня бензин... Приезжай, заправлю.... \n'

In [62]:
get_answer('Почему мало платят?')

'Ну это как Вы на это смотрите, если для Вас жизнь на Земле - это единственная жизнь , то для Вас этот вопрос актуален. Для меня жизнь на Земле - это временный период набираться знаний , мудрости , и к старости я отношусь спокойно , мозгами я молодая. Тело стареет , так это всего лишь одежка данная мне Природой на определенный срок . Я считаю , что человек платит старостью , ведь взамен он получает больше - радость пребывание на Земле , знания , мудрость - а за все мы должны чем то заплатить . Вот старость - это и есть плата . Это мое видение , понимание .. \n'

Здесь адекватные ответы, конечно, сложно получить. Я пробовал по разному предобрабатывать текст и получается примерно одно и то же. Думаю, что дело в самом датасете. Какого-то эффекта от взвешивания tf-idf не обнаружил.

In [38]:
import warnings
warnings.filterwarnings("ignore")

updater = Updater(token=token)
dispatcher = updater.dispatcher

In [39]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Бот готов ответить на ваши вопросы!')

In [40]:
def textMessage(bot, update):
    answer = get_answer(update.message.text)
    bot.send_message(chat_id=update.message.chat_id, text=answer)

In [41]:
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

Разворачивать на хостинге не стал пока. Протестировал бота, запустив ячейку. В принципе все работает.